In [1]:
import ee 
import geemap
import plotly.graph_objects as go
from ipyleaflet import WidgetControl
from ipywidgets import HTML

In [2]:
# Inicializar o Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-vl99956018')

# Parâmetros de data e região
pre_start_date = '2025-01-01'  # Período antes do fogo
pre_end_date = '2025-01-05'
pos_start_date = '2025-01-06'  # Período após o fogo
pos_end_date = '2025-01-16'

# Região de interesse (ROI)
roi = ee.Geometry.Point(-118.532875, 34.092615).buffer(15000)  # Área de 15km ao redor do ponto

# Função para processar coleção (NDVI e NBR)
def process_collection(start_date, end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select('B8', 'B12', 'B4', 'B3', 'B2')

    image = collection.median().clip(roi)
    return image

# Processar coleções pré e pós incêndio
pre_image = process_collection(pre_start_date, pre_end_date)
pos_image = process_collection(pos_start_date, pos_end_date)

# Visualização RGB
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  # Valor único para todas as bandas
    'gamma': 1.2
}

# Criar o mapa
Map = geemap.Map(center=[34.092615, -118.532875], zoom=13)

# Configurar o split map corretamente
left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Pré-incêndio')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'Pós-incêndio')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Adicionar controle de camadas para alternar entre visualizações
Map.addLayer(pre_image, rgb_vis, 'Pré-incêndio', False)
Map.addLayer(pos_image, rgb_vis, 'Pós-incêndio', False)

def export_to_drive(image, description, folder_name='GEE_exports'):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder_name,
        fileNamePrefix=description.lower().replace(' ', '_'),
        scale=10,  # Sentinel-2 resolution
        region=roi,
        fileFormat='GeoTIFF',
        maxPixels=1e10
    )
    task.start()
    print(f'Started export task for {description}. Check your Google Drive folder "{folder_name}" when complete.')

export_to_drive(pre_image, 'Pre-fire NDVI GEOTIFF image')
export_to_drive(pos_image, 'Post-fire NDVI GEOTIFF image')

# Exibir o mapa
Map

Started export task for Pre-fire NDVI GEOTIFF image. Check your Google Drive folder "GEE_exports" when complete.
Started export task for Post-fire NDVI GEOTIFF image. Check your Google Drive folder "GEE_exports" when complete.


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [13]:
# Inicializar o Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-vl99956018')

# Parâmetros de data e região
pre_start_date = '2025-01-01'
pre_end_date = '2025-01-05'
pos_start_date = '2025-01-06'
pos_end_date = '2025-01-16'

# Região de interesse (ROI)
roi = ee.Geometry.Point(-118.532875, 34.092615).buffer(15000)

# Função para calcular NDVI
def get_ndvi_collection(start_date, end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(['B8', 'B4'])
    
    ndvi_collection = collection.map(
        lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    )
    
    return ndvi_collection.median().clip(roi)

# Obter imagens NDVI
pre_ndvi = get_ndvi_collection(pre_start_date, pre_end_date)
pos_ndvi = get_ndvi_collection(pos_start_date, pos_end_date)

# Configuração de visualização
ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

# Criar o mapa
Map = geemap.Map(center=[34.092615, -118.532875], zoom=13)

# Adicionar split map
Map.split_map(
    left_layer=geemap.ee_tile_layer(pre_ndvi, ndvi_vis, 'Antes do Incêndio'),
    right_layer=geemap.ee_tile_layer(pos_ndvi, ndvi_vis, 'Depois do Incêndio')
)

# Adicionar legendas de texto em preto com fundo branco
antes_html = HTML('''<div style="position: absolute; top: 10px; left: 10px; z-index: 1000; 
                    background-color: white; padding: 5px; border-radius: 5px;
                    font-weight: bold; color: black;">ANTES</div>''')
                    
depois_html = HTML('''<div style="position: absolute; top: 10px; right: 10px; z-index: 1000; 
                     background-color: white; padding: 5px; border-radius: 5px;
                     font-weight: bold; color: black;">DEPOIS</div>''')

# Adicionar controles ao mapa
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

def export_to_drive(image, description, folder_name='GEE_exports'):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder_name,
        fileNamePrefix=description.lower().replace(' ', '_'),
        scale=10,  # Sentinel-2 resolution
        region=roi,
        fileFormat='GeoTIFF',
        maxPixels=1e10
    )
    task.start()
    print(f'Started export task for {description}. Check your Google Drive folder "{folder_name}" when complete.')

export_to_drive(pre_ndvi, 'Pre-fire NDVI GEOTIFF image')
export_to_drive(pos_ndvi, 'Post-fire NDVI GEOTIFF image')


# Exibir o mapa
Map

Started export task for Pre-fire NDVI GEOTIFF image. Check your Google Drive folder "GEE_exports" when complete.
Started export task for Post-fire NDVI GEOTIFF image. Check your Google Drive folder "GEE_exports" when complete.


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [2]:
# Inicializar o Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-vl99956018')

# Parâmetros de data e região
pre_start_date = '2025-01-01'
pre_end_date = '2025-01-05'
pos_start_date = '2025-01-06'
pos_end_date = '2025-01-16'

# Região de interesse (ROI)
roi = ee.Geometry.Point(-118.532875, 34.092615).buffer(15000)

# Função para calcular NDVI
def get_nbr_collection(start_date, end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(['B8', 'B12'])
    
    nbr_collection = collection.map(
        lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')
    )
    
    return nbr_collection.median().clip(roi)

# Obter imagens NDVI
pre_nbr = get_nbr_collection(pre_start_date, pre_end_date)
pos_nbr = get_nbr_collection(pos_start_date, pos_end_date)

# Configuração de visualização
nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
}
# Criar o mapa
Map = geemap.Map(center=[34.092615, -118.532875], zoom=13)

# Adicionar split map
Map.split_map(
    left_layer=geemap.ee_tile_layer(pre_nbr, nbr_vis, 'Antes do Incêndio'),
    right_layer=geemap.ee_tile_layer(pos_nbr, nbr_vis, 'Depois do Incêndio')
)

# Adicionar legendas de texto em preto com fundo branco
antes_html = HTML('''<div style="position: absolute; top: 10px; left: 10px; z-index: 1000; 
                    background-color: white; padding: 5px; border-radius: 5px;
                    font-weight: bold; color: black;">ANTES</div>''')
                    
depois_html = HTML('''<div style="position: absolute; top: 10px; right: 10px; z-index: 1000; 
                     background-color: white; padding: 5px; border-radius: 5px;
                     font-weight: bold; color: black;">DEPOIS</div>''')

# Adicionar controles ao mapa
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

# def export_to_drive(image, description, folder_name='GEE_exports'):
#     task = ee.batch.Export.image.toDrive(
#         image=image,
#         description=description,
#         folder=folder_name,
#         fileNamePrefix=description.lower().replace(' ', '_'),
#         scale=10,  # Sentinel-2 resolution
#         region=roi,
#         fileFormat='GeoTIFF',
#         maxPixels=1e10
#     )
#     task.start()
#     print(f'Started export task for {description}. Check your Google Drive folder "{folder_name}" when complete.')

# export_to_drive(pre_nbr, 'Pre-fire NBR GEOTIFF image')
# export_to_drive(pos_nbr, 'Post-fire NBR GEOTIFF image')

# Exibir o mapa
Map

Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [3]:
import ee
import geemap
import datetime
import plotly.graph_objects as go
import os


# Corrigindo data final (fevereiro só tem 29 dias em 2025 porque é ano bissexto)
start_date = '2024-11-01'
end_date = '2025-04-01'

# Região de interesse
roi = ee.Geometry.Point(-118.532875, 34.092615).buffer(1000)

def get_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date, end_date):
    # Carrega coleção Sentinel-2
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterDate(start_date, end_date)
                    .filterBounds(roi)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                    .map(get_ndvi)
                    .map(get_nbr)
                    .select(['NDVI', 'NBR']))

    # Função para extrair NDVI e NBR médios e data como Feature
    def reduce_img(img):
        stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        )
        return ee.Feature(None, {
            'date': img.date().format('YYYY-MM-dd'),
            'NDVI': stats.get('NDVI'),
            'NBR': stats.get('NBR')
        })

    # Aplica a função e cria FeatureCollection
    features = collection.map(reduce_img)

    # Extrai listas
    stats_list = features.aggregate_array('date').getInfo()
    ndvi_values = features.aggregate_array('NDVI').getInfo()
    nbr_values = features.aggregate_array('NBR').getInfo()

    # Converter datas para objetos datetime
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

    # Plot NDVI and NBR curves in the same figure
    fig = go.Figure()
    
    # Add NDVI trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=ndvi_values,
        mode='lines+markers',
        name='NDVI',
        line=dict(color='green', width=2),
        marker=dict(size=8, color='green'),
        yaxis='y1'
    ))
    
    # Add NBR trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=nbr_values,
        mode='lines+markers',
        name='NBR',
        line=dict(color='red', width=2),
        marker=dict(size=8, color='red'),
        yaxis='y1'
    ))

    # Add vertical line at 2025-01-07
    fig.add_vline(
        x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
        line_width=2,
        line_dash="dash",
        line_color="blue",
        annotation_text="First Fire detected",
        annotation_position="top right"
    )
    # Update layout (rest remains the same)
    fig.update_layout(
        title='NDVI and NBR Temporal Curves',
        xaxis_title='Date',
        yaxis_title='Index Value',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True, title='Index Value'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_white',
        hovermode='x unified'
    )

    # Save plot
    os.makedirs("results", exist_ok=True)
    fig.write_html("results/ndvi_nbr_plot.html")
    print("Gráfico salvo como 'results/ndvi_nbr_plot.html'")

# Run the function
ndvi_nbr_plot(start_date, end_date)

Gráfico salvo como 'results/ndvi_nbr_plot.html'
